In [1]:
import fitz  # PyMuPDF
import faiss
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document
from langchain.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os
from langchain.vectorstores import Chroma 
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma 
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.chains import RetrievalQA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel, AutoTokenizer
from langchain.document_loaders import TextLoader
import torch
from sentence_transformers import CrossEncoder

C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
def setup_environment():
    import sys
    sys.path.append('C:\\gitworkspace\\aimldemo\\jupyterworkapce')
    import stratup_env_setup
    stratup_env_setup.set_env()

In [3]:
setup_environment()

In [4]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [5]:
llm_mistral = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2")

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24920\936897557.py:1: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm_mistral = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\jimmy\.cache\huggingface\token
Login successful


In [6]:
chunk_size=1000

In [7]:
pdf_file_path = "doc_inputs/Form_2287.pdf"
#pdf_file_path = "doc_inputs/Form_2287.txt"
#pdf_file_path = "doc_inputs/John-NIV.pdf"
#pdf_file_path = 'doc_inputs/falcon-users-guide-2021-09.pdf'

In [8]:
#query = "In the event of default under the Third Party Loan, what will CDC or SBA may do?"
#query = 'what does this document say about Authority to Execute Agreement?'
#query = 'What does this document say about waiver of Right to Indemnification by SBA or CDC.'
#query = 'where can I find the standard price for Falcon 9 and Falcon Heavy launch services?'
#query = 'SpaceX requires that customers verify the mass properties of their system through measurement before shipping it to the launch site.Is this true or false?'
#query = 'What does this document say about Maturity and Balloon Payments?'
query = 'Any balloon payment for the Third Party Loan must be disclosed to whom?'
#query = 'Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term'
#query = 'Hello How are you?'

In [9]:
top_n=10

# Mistral, Chroma DB and PyPDFLoader

In [10]:
query

'Any balloon payment for the Third Party Loan must be disclosed to whom?'

In [11]:
if pdf_file_path.endswith(".pdf"):
    # Load PDF file
    print('Loading PDF file')
    pdf_loader = PyPDFLoader(pdf_file_path)
    documents = pdf_loader.load()
elif pdf_file_path.endswith(".txt"):
    # Load TXT file
    print('Loading text file')
    txt_loader = TextLoader(pdf_file_path)
    documents = txt_loader.load()
else:
    raise ValueError("Unsupported file format. Please provide a .pdf or .txt file.")

Loading PDF file


In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=200)

In [13]:
chunks = text_splitter.split_documents(documents)

In [14]:
type(chunks[0])

langchain_core.documents.base.Document

In [15]:
embeddings = OpenAIEmbeddings()

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24920\2497576997.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [16]:
# Get embeddings for the query
query_embedding = embeddings.embed_query(query)

In [17]:
# Print top 3 similar chunks before persisting into DB
# Calculate the similarity for each chunk and store it along with the chunk
similarities = []
for i, chunk in enumerate(chunks):
    # Get embedding for the chunk
    chunk_embedding = embeddings.embed_query(chunk.page_content)
    
    # Calculate cosine similarity between query and chunk
    similarity = cosine_similarity([query_embedding], [chunk_embedding])[0][0]
    
    # Store similarity, chunk content, and metadata for sorting
    similarities.append((similarity, chunk.page_content, chunk.metadata))
    
# Sort the chunks based on similarity in descending order and get the top 3
top_chunks = sorted(similarities, key=lambda x: x[0], reverse=True)[:top_n]

# Print the top 3 chunks
for i, (similarity, content, metadata) in enumerate(top_chunks, start=1):
    page_number = metadata.get('page', 'N/A')
    print(f"Top Chunk {i}: Page number: {page_number}")
    print(content)
    print(f"Cosine Similarity: {similarity * 100:.4f}")
    print("#" * 100)  # Optional separator for readability

Top Chunk 1: Page number: 2
exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third 
Party Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the 
Third Party Loan prior to maturity unless the Third Party L oan is in material  default .   
 
 e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at 
least 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the 
504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an 
overall loan maturity must be calculated , taking into account the amounts and maturities  of each 
loan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to 
SBA and approved at application or subsequently a pproved by SBA .  
 
 f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rate
Cosine Similarity: 85.1362
########

In [18]:
###################################################

* Re ranking the top n results based on CrossEncoder

In [19]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [20]:
input_pairs = [(query, content) for _, content, _ in top_chunks]

In [21]:
cross_encoder_scores = cross_encoder.predict(input_pairs)

In [22]:
# Print the CrossEncoder scores along with the chunk details
print("\nCrossEncoder Scores for Top Chunks:")
for i, (score, (similarity, content, metadata)) in enumerate(zip(cross_encoder_scores, top_chunks), start=1):
    page_number = metadata.get('page', 'N/A')
    print(f"Top Chunk {i}: Page number: {page_number}")
    print(content)
    print(f"Cosine Similarity: {similarity * 100:.4f}")
    print(f"CrossEncoder Score: {score:.4f}")
    print("#" * 100)  # Optional separator for readability


CrossEncoder Scores for Top Chunks:
Top Chunk 1: Page number: 2
exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third 
Party Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the 
Third Party Loan prior to maturity unless the Third Party L oan is in material  default .   
 
 e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at 
least 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the 
504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an 
overall loan maturity must be calculated , taking into account the amounts and maturities  of each 
loan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to 
SBA and approved at application or subsequently a pproved by SBA .  
 
 f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rat

In [23]:
######################### Cosine similarity with BERT Higgingface 

In [24]:
from sentence_transformers import SentenceTransformer

In [25]:
bert_model = SentenceTransformer('bert-base-uncased')

No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


In [26]:
query

'Any balloon payment for the Third Party Loan must be disclosed to whom?'

In [27]:
query_embedding = bert_model.encode(query, convert_to_tensor=True)
query_embedding = query_embedding if isinstance(query_embedding, list) else query_embedding.cpu().numpy()

In [28]:
# Calculate the similarity for each chunk and store it along with the chunk
similarities = []
for i, chunk in enumerate(chunks):
    # Get BERT embedding for the chunk and convert to a NumPy array
    chunk_embedding = bert_model.encode(chunk.page_content, convert_to_tensor=True)
    chunk_embedding = chunk_embedding if isinstance(chunk_embedding, list) else chunk_embedding.cpu().numpy()
    
    # Calculate cosine similarity between query and chunk embeddings
    similarity = cosine_similarity([query_embedding], [chunk_embedding])[0][0]

    # Store similarity, chunk content, and metadata for sorting
    similarities.append((similarity, chunk.page_content, chunk.metadata))

In [29]:
# Sort the chunks based on similarity in descending order and get the top 3
top_chunks = sorted(similarities, key=lambda x: x[0], reverse=True)[:top_n]

In [30]:
# Print the top 3 chunks
for i, (similarity, content, metadata) in enumerate(top_chunks, start=1):
    page_number = metadata.get('page', 'N/A')
    print(f"Top Chunk {i}: Page number: {page_number}")
    print(content)
    print(f"Cosine Similarity: {similarity * 100:.4f}")
    print("#" * 100)  # Optional separator for readability

Top Chunk 1: Page number: 2
liquidati on, or has insufficient value to justify the cost of collection, then the Third Party Lender is not 
required to liquidate such collateral , provided the Third Party Lender notifies CDC/SBA.   
 
7. Notice of Default under the Third Party Loan.    Within thirty  (30) days after the expiration of any 
cure period  for any  continuing material default of the  Third Party Loan or Third Party Lender Lien, Third 
Party Lender must provide written notice (referencing SBA’s loan number for the 504 Loan) of the default to  
CDC  and SBA .  At least sixty 60 days prior to any legal proceeding s against or liquidation of t he Common 
Collateral  (not including sending a demand letter) , Third Party Lender must provide SBA with written notice 
of its intent to do so. 
 
8. Limitation on  Defa ult Interest Rate.    Third Party Lender may not escalate the rate of interest upon 
default to a rate greater than the maximum rate published by SBA in the Federal Regi

In [31]:
######################### use multi-qa-distilbert-cos-v1

In [32]:
bert_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [33]:
query_embedding = bert_model.encode(query)
query_embedding = query_embedding if isinstance(query_embedding, list) else query_embedding

In [34]:
# Calculate the similarity for each chunk and store it along with the chunk
similarities = []
for i, chunk in enumerate(chunks):
    # Get BERT embedding for the chunk and convert to a NumPy array
    chunk_embedding = bert_model.encode(chunk.page_content)
    chunk_embedding = chunk_embedding if isinstance(chunk_embedding, list) else chunk_embedding

    # Calculate cosine similarity between query and chunk embeddings
    similarity = cosine_similarity([query_embedding], [chunk_embedding])[0][0]

    # Store similarity, chunk content, and metadata for sorting
    similarities.append((similarity, chunk.page_content, chunk.metadata))

In [35]:
# Sort the chunks based on similarity in descending order and get the top n
top_chunks = sorted(similarities, key=lambda x: x[0], reverse=True)[:top_n]

In [37]:
# Print the top n chunks
for i, (similarity, content, metadata) in enumerate(top_chunks, start=1):
    page_number = metadata.get('page', 'N/A')
    print(f"Top Chunk {i}: Page number: {page_number}")
    print(content)
    print(f"Cosine Similarity: {similarity * 100:.4f}")
    print("#" * 100)  # Optional separator for readability

Top Chunk 1: Page number: 1
be open- ended.  After completion of the Project , the Third Party Lender may not make future 
advances under the Third P arty Loan except for reasonable cost s of collection, ma intenance, and 
protection of the Third Party Loan  and Third Party  Lender  Lien.  
 
 b. No Early Call or Demand Provisions.    Third Party Lender agrees that  documents  
evidencing the Third Party Loan and the Third Party Lender Lien do not  contain an early  call feature 
or any provision which allows Third Party Lender   to make demand under the Third Party Lender 
Loan other than when there is a material default under the terms of its Third Party Loan documents ,  
which shall include , but not be limited to, failure to make timely payments  on the Third Party Loan ,
Cosine Similarity: 55.5278
####################################################################################################
Top Chunk 2: Page number: 0
Term of Third Party Loan :  ____________________________

In [50]:
############################################

In [51]:
vector_store = Chroma.from_documents(chunks, embeddings)

In [52]:
retriever=vector_store.as_retriever()

In [53]:
retrieved_docs = retriever.get_relevant_documents(query)

In [54]:
type(retrieved_docs[0])

langchain_core.documents.base.Document

In [55]:
unique_docs = set()

In [56]:
for doc in retrieved_docs:
    unique_docs.add(doc.page_content)  # Replace 'page_content' with a unique identifier if available   


In [57]:
print(f"Total Retrieved Documents: {len(retrieved_docs)}")
print(f"Unique Documents Retrieved: {len(unique_docs)}")

Total Retrieved Documents: 4
Unique Documents Retrieved: 2


In [58]:
# recalcualte openAI query embedding
query_embedding = embeddings.embed_query(query)

In [59]:
# Print the documents and their cosine similarity with the query
for i, d in enumerate(retrieved_docs):
    # Get the embedding for the current document chunk
    doc_embedding = embeddings.embed_query(d.page_content)
    
    # Compute the cosine similarity between the query and the document
    similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]
    
    # Print the document content and the similarity score
    print(f"Document {i + 1}:Page number: {d.metadata['page']}:  {d.page_content}")
    print(f"Cosine Similarity: {similarity*100:.4f}")
    print("#" * 100)  # Optional: To visually separate the chunks in the output

Document 1:Page number: 2:  exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third 
Party Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the 
Third Party Loan prior to maturity unless the Third Party L oan is in material  default .   
 
 e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at 
least 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the 
504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an 
overall loan maturity must be calculated , taking into account the amounts and maturities  of each 
loan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to 
SBA and approved at application or subsequently a pproved by SBA .  
 
 f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rate
Cosine Similarity: 85.1362
########

In [60]:
for i, d in enumerate(retrieved_docs):
    print(f"Document {i + 1}: {d.page_content}")
    print("#" * 100)  # Optional: To visually separate the chunks in the output

Document 1: exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third 
Party Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the 
Third Party Loan prior to maturity unless the Third Party L oan is in material  default .   
 
 e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at 
least 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the 
504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an 
overall loan maturity must be calculated , taking into account the amounts and maturities  of each 
loan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to 
SBA and approved at application or subsequently a pproved by SBA .  
 
 f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rate
###################################################

In [61]:
type(retrieved_docs)

list

In [62]:
retrieved_docs

[Document(metadata={'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}, page_content='exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third \nParty Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party L oan is in material  default .   \n \n e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at \nleast 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the \n504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an \noverall loan maturity must be calculated , taking into account the amounts and maturities  of each \nloan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .  \n \n f. Reasonable Interest Rate.   The Third Party Loan

In [63]:
retrieved_doc = retrieved_docs[0] 

In [64]:
retrieved_doc

Document(metadata={'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}, page_content='exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third \nParty Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party L oan is in material  default .   \n \n e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at \nleast 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the \n504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an \noverall loan maturity must be calculated , taking into account the amounts and maturities  of each \nloan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .  \n \n f. Reasonable Interest Rate.   The Third Party Loan 

In [65]:
retrieved_text = retrieved_doc.page_content

In [66]:
retrieved_metadata = retrieved_doc.metadata

In [67]:
retrieved_text

'exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third \nParty Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party L oan is in material  default .   \n \n e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at \nleast 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the \n504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an \noverall loan maturity must be calculated , taking into account the amounts and maturities  of each \nloan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .  \n \n f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rate'

In [68]:
retrieved_metadata

{'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}

In [69]:
embeddings = OpenAIEmbeddings()
llm_gpt = ChatOpenAI(model='gpt-4', temperature=1)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24920\2899348281.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_gpt = ChatOpenAI(model='gpt-4', temperature=1)


* Use ContextualCompressionRetriever

In [70]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [71]:
compressor = LLMChainExtractor.from_llm(llm_gpt)

In [72]:
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

In [73]:
docs_compressed = compression_retriever.get_relevant_documents(query)

In [74]:
docs_compressed

[Document(metadata={'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}, page_content='Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .'),
 Document(metadata={'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}, page_content='Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .')]

In [75]:
docs_compressed[0]

Document(metadata={'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}, page_content='Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .')

In [76]:
system_prompt = (
    "You are an assistant for question-answering tasks. Answer the following question based only on the context provided. Do not use any external knowledge."
     "\n\n"
    "{context}"
)

In [77]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [78]:
question_answer_chain = create_stuff_documents_chain(llm_gpt, prompt)

In [79]:
inputs = {
    "context": [retrieved_docs[0]],  # use only the first elemnt from retrieved_docs
    "input": query  # Use the query as the input for the human message
}

In [80]:
response = question_answer_chain.invoke(inputs)

In [81]:
response

'Any balloon payment for the Third Party Loan must be disclosed to SBA.'

In [82]:
type(response)

str

In [83]:
input_text = f"""
Context: {retrieved_text}

Instruction: Answer the following question based only on the context provided. Do not use any external knowledge.

Question: {query}

Answer:
"""

In [84]:
input_text

'\nContext: exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third \nParty Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party L oan is in material  default .   \n \n e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at \nleast 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the \n504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an \noverall loan maturity must be calculated , taking into account the amounts and maturities  of each \nloan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .  \n \n f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rate\n\nInstruction: Answer the following que

In [85]:

response_gpt = llm_gpt.predict(input_text)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24920\2746106821.py:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_gpt = llm_gpt.predict(input_text)


In [86]:
response_gpt

'Any balloon payment for the Third Party Loan must be disclosed to SBA.'

In [87]:
#chain_mistral = RetrievalQA.from_chain_type(llm_mistral, retriever=retriever )
#response_mistral = chain_mistral.run(input_text)
response_mistral = llm_mistral.predict(input_text)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24920\3110740755.py:3: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_mistral = llm_mistral.predict(input_text)


In [89]:
response_mistral

'The context states that any balloon payment for the Third Party Loan must be clearly identified and disclosed to SBA.'

In [90]:
######################################################################

In [91]:
# Highlight using cosine similarity
# Step 1: Extract content to be highlighted from the first retrieved document
content_to_highlight = retrieved_text

# Step 2: Open the original PDF
pdf_document = fitz.open(pdf_file_path)

# Step 3: Initialize variables for tracking the best match
best_page = None
best_match_rects = None
highest_similarity = 0
best_page_number = None  # Initialize best_page_number her

# Step 4: Iterate through each page in the PDF
for page_num in range(len(pdf_document)):
    page = pdf_document[page_num]
    page_text = page.get_text()

    # Step 5: Split the page text into smaller chunks
    page_chunks = [page_text[i:i+chunk_size] for i in range(0, len(page_text), chunk_size)]

    # Step 6: Perform text similarity search using TF-IDF and cosine similarity
    vectorizer = TfidfVectorizer().fit_transform([content_to_highlight] + page_chunks)
    similarities = cosine_similarity(vectorizer[0:1], vectorizer[1:]).flatten()

    # Step 7: Find the chunk with the highest similarity score
    max_sim_index = similarities.argmax()
    max_sim = similarities[max_sim_index]

    # Step 8: Update the best match if this page has a higher similarity score
    if max_sim > highest_similarity:
        highest_similarity = max_sim
        best_page = page
        best_chunk = page_chunks[max_sim_index]
        best_page_number = page_num + 1  # Store the 1-based page number

        # Locate the matching text in the page
        best_match_rects = best_page.search_for(best_chunk)

In [92]:
# Step 9: Highlight the best match
if best_page and best_match_rects:
    for rect in best_match_rects:
        best_page.add_highlight_annot(rect)

In [93]:
# Step 10: Print the best page number
if best_page_number is not None:
    print(f"The best match was found on page number: {best_page_number}")

The best match was found on page number: 3


In [94]:
# Step 10: Save the modified PDF
highlighted_pdf_path = "doc_outputs/highlighted_file_chroma.pdf"
pdf_document.save(highlighted_pdf_path)

In [95]:
# Close the PDF document
pdf_document.close()

* Query for meta data

In [ ]:
metadata_field_info = [
    AttributeInfo(
        name="page",
        description="The page number within teh document.Starting with 0",
        type="integer",
    ),
    AttributeInfo(
        name="source",
        description="The name of the document as a file name",
        type="string",
    ),    
]

In [ ]:
document_content_description = "Content of a specific page"

In [ ]:
retriever = SelfQueryRetriever.from_llm(
    llm_gpt,
    vector_store,
    document_content_description,
    metadata_field_info,
)

In [ ]:
retriever.invoke("I want the page number 1")

# Chroma storeage using local BERT Embedding <span style="color:red;">(Embedding result is very poor)</span>

In [ ]:
import uuid 

In [ ]:
query = """Subordination of 504 Loan.  CDC agrees to make the 504 Loan to the Borrower , subject to SBA’s 
approval , and accept a junior and subordinate lien position in the Com mon Collateral upon the condition s that 
Third Party Lender executes this Agreement and disburses the  Third Party Loan  according to the terms 
represented to CDC and SBA"""

In [ ]:
pdf_loader = PyPDFLoader(pdf_file_path)

In [ ]:
documents = pdf_loader.load()

In [ ]:
local_directory = './bert_downloads'

In [ ]:
# Step 2: Split the documents into chunks
chunk_size = 1000  # Adjust this size as needed
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
for i, chunk in enumerate(chunks):
    # Get the chunk content
    content = chunk.page_content
    
    # Get the metadata (e.g., page number)
    metadata = chunk.metadata
    page_number = metadata.get('page', None)
    
    # Print the chunk content and page number
    print(f"Chunk {i + 1}:")
    print(f"Page Number: {page_number+1}")
    print(f"Content: {content}")
    print("#" * 100)  # Optional: To visually separate the chunks in the output

In [ ]:
# Step 3: Load the BERT model and tokenizer from the local directory
tokenizer = AutoTokenizer.from_pretrained(local_directory, local_files_only=True)
model = AutoModel.from_pretrained(local_directory, local_files_only=True)

In [ ]:
# Function to get embeddings using BERT
def get_embedding(text):
    # Tokenize and encode the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the last hidden state as the embedding
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

In [ ]:
# Step 4: Create embeddings for the chunks
texts = [chunk.page_content for chunk in chunks]
embeddings = [get_embedding(text).tolist() for text in texts]  # Convert numpy arrays to lists
ids = [str(uuid.uuid4()) for _ in texts]  # Generate unique IDs for each chunk

In [ ]:
# Step 5: Initialize the Chroma vector store with a specified collection name and persist directory
vector_store = Chroma(collection_name="my_collection", persist_directory="chroma_db")

In [ ]:
embeddings

In [ ]:
# Step 6: Manually add the texts and embeddings to the vector store using upsert
vector_store._collection.upsert(
    documents=texts,
    embeddings=embeddings,
    metadatas=[chunk.metadata for chunk in chunks],
    ids=ids,  # Provide the generated unique IDs
)

In [ ]:
# Step 7: Retrieve relevant documents based on the query manually
query_embedding = get_embedding(query).tolist()

In [ ]:
query_embedding

In [ ]:
top_k=4

In [ ]:
# Function to compute cosine similarity and retrieve top matches
# Compute similarities for all chunks
similarities = []
for i, (embedding, text) in enumerate(zip(embeddings, texts)):
    similarity = cosine_similarity([query_embedding], [embedding])[0][0]
    similarities.append((similarity, text))

# Sort by similarity in descending order and select top_k chunks
top_chunks = sorted(similarities, key=lambda x: x[0], reverse=True)[:top_k]

# Print the top chunks with their similarity scores
for i, (similarity, content) in enumerate(top_chunks, start=1):
    print(f"Top Chunk {i}:")
    print(content)
    print(f"Cosine Similarity: {similarity * 100:.4f}")
    print("#" * 100)  # Separator for readability


In [ ]:
# Step 7: Retrieve the top 3 relevant documents from Chroma based on the query
top_k = 3  # Number of top documents to retrieve
retrieved_docs = vector_store._collection.query(
    query_embeddings=[query_embedding],
    n_results=top_k,
    include=['documents', 'embeddings', 'metadatas']
)

In [ ]:
unique_docs = set() 

In [ ]:
# Assuming each retrieved document has an ID or unique content
for doc in retrieved_docs['documents']:
    # Use either a unique ID or the content itself for uniqueness check
    unique_docs.add(doc[0])  # Replace 'id' with the unique identifier field name
print(f"Total Retrieved Documents: {len(retrieved_docs['documents'])}")
print(f"Unique Documents Retrieved: {len(unique_docs)}")

In [ ]:
unique_docs

In [ ]:
# Extract the documents, embeddings, and metadata from the query results
retrieved_texts = [doc for doc in retrieved_docs['documents'][0]]
retrieved_embeddings = [embedding for embedding in retrieved_docs['embeddings'][0]]
retrieved_metadata = [meta for meta in retrieved_docs['metadatas'][0]]

In [ ]:
# Step 8: Calculate the cosine similarity between the query and each of the retrieved document embeddings
similarities = [
    cosine_similarity([query_embedding], [embedding])[0][0] for embedding in retrieved_embeddings
]

In [ ]:
# Step 9: Print the top 3 documents, their similarity scores, and page numbers
for i, (similarity, content, metadata) in enumerate(zip(similarities, retrieved_texts, retrieved_metadata)):
    # Extract the page number from the metadata
    page_number = metadata.get('page', None)
    
    # Print the document content, page number, and similarity score
    print(f"Top {i + 1} Document: Page {page_number+1}")
    print(f"Cosine Similarity: {similarity*100:.4f}")
    print(f"Content: {content}")
    print("#" * 100)  # Optional: To visually separate the chunks in the output

# Using Mistral <span style="color:red;">(PDF extraction and chunking not good)</span>

In [ ]:
doc = fitz.open(pdf_file_path)

In [ ]:
type(doc)

In [ ]:
metadata = []
current_chunk = ""
current_bbox = None  # To track the bounding box for the chunk
current_page_num = None

for page_num in range(len(doc)):
    page = doc[page_num]
    text_blocks = page.get_text("blocks")

    for block in text_blocks:
        text = block[4].strip()
        bbox = block[:4]  # Bounding box: (x0, y0, x1, y1)
        
        if text:  # Ensure the block contains text
            if len(current_chunk) + len(text) <= chunk_size:
                # Accumulate text in the current chunk
                if not current_chunk:  # Start a new chunk
                    current_bbox = bbox
                    current_page_num = page_num
                current_chunk += " " + text
            else:
                # Store the current chunk in metadata
                metadata.append({
                    'page_num': current_page_num,
                    'bbox': current_bbox,
                    'text': current_chunk.strip()
                })
                # Start a new chunk with the current block
                current_chunk = text
                current_bbox = bbox
                current_page_num = page_num

    # After processing all blocks on the page, check if there's a remaining chunk
    if current_chunk:
        metadata.append({
            'page_num': current_page_num,
            'bbox': current_bbox,
            'text': current_chunk.strip()
        })
        current_chunk = ""
        current_bbox = None

# After all pages are processed, ensure any remaining text is stored
if current_chunk:
    metadata.append({
        'page_num': current_page_num,
        'bbox': current_bbox,
        'text': current_chunk.strip()
    })


In [ ]:
metadata

In [ ]:
# Initialize the embedding model using HuggingFaceEmbeddings with "mistral-7b"
embedding_model = HuggingFaceEmbeddings(model_name="bert-base-uncased")

In [ ]:
d = len(embedding_model.embed_query("test"))  # Determine the dimension

In [ ]:
d

In [ ]:
faiss_index = faiss.IndexFlatL2(d)
docstore = InMemoryDocstore()
vector_store = FAISS(index=faiss_index, docstore=docstore, index_to_docstore_id={}, embedding_function=embedding_model.embed_query)


In [ ]:
# Step 4: Store each page's text and metadata in the FAISS vector database
for i, item in enumerate(metadata):
    text = item['text']
    embedding = embedding_model.embed_query(text)
    
    # Create a Document object with the text and metadata
    doc_obj = Document(page_content=text, metadata={'page_num': item['page_num'], 'bbox': item['bbox']})
    
    # Add the embedding and corresponding document to FAISS vector store
    vector_store.add_texts([text], [doc_obj], embeddings=[embedding])

In [ ]:
query

In [ ]:
# Set up the retriever
retriever = vector_store.as_retriever()
retrieved_docs = retriever.get_relevant_documents(query)

In [ ]:
# Inspect what was retrieved
for i, d in enumerate(retrieved_docs):
    print(f"Document {i + 1}: {d.page_content}")
    print("#" * 100)  # Optional: To visually separate the chunks in the output

In [ ]:
#retrieved_text = " ".join([d.page_content for d in retrieved_docs])
# consider only first retrived text
retrieved_text = retrieved_docs[0].page_content 

In [ ]:
retrieved_text

* FAISS DB retrieves wrong text. Try with Chroma DB as below

In [ ]:
# Use Chroma DB
# Initialize the OpenAI embedding model
embedding_model = OpenAIEmbeddings()
vector_store = Chroma(embedding_function=embedding_model, persist_directory="chroma_db")

# List to hold Document objects
documents = []

for item in metadata:
    text = item['text']
    bbox_str = str(item['bbox'])  # Convert bbox tuple to string
    doc_obj = Document(page_content=text, metadata={'page_num': item['page_num'], 'bbox': bbox_str})
    documents.append(doc_obj)



# Step 4: Add documents to the Chroma vector store
vector_store.add_documents(documents)
    
# Don't forget to persist the vector store
#vector_store.persist()

In [ ]:
#query_embedding = embedding_model.embed_query(query)

In [ ]:
results = vector_store.similarity_search_with_score(query, k=5)  # k defines how many results you want

In [ ]:
# Display the results
for result, score in results:
    print(f"Score: {score}")
    print(f"Page Number: {result.metadata['page_num']}")
    print(f"Bounding Box: {result.metadata['bbox']}")
    print(f"Text: {result.page_content}\n")

In [ ]:
first_result = results[0][0]

In [ ]:
bbox = first_result.metadata['bbox']
page_num = first_result.metadata['page_num']
first_text = first_result.page_content

In [ ]:
bbox

In [ ]:
page_num

In [ ]:
retrieved_text = first_text

In [ ]:
retrieved_text

In [ ]:
input_text = f"""
Context: {retrieved_text}

Instruction: Answer the following question based only on the context provided. Do not use any external knowledge.

Question: {query}

Answer:
"""

In [ ]:
# Prepare the input text for the LLM
#input_text = f"Context: {retrieved_text}\n\nQuestion: {query}\nAnswer:"

In [ ]:
input_text

In [ ]:
# Use the LLM to generate an answer
answer = llm_mistral(input_text)

In [ ]:
answer

In [ ]:
highlighted_pdf_path = 'doc_outputs/highlighted_file_faiss.pdf'

In [ ]:
# Generate an embedding for the LLM's response
response_embedding = embedding_model.embed_query(answer)

In [ ]:
# Perform cosine similarity search using LangChain's FAISS index
similar_docs = vector_store.similarity_search_with_score(answer, k=1)

In [ ]:
# Retrieve the best-matching section
best_match = similar_docs[0][0].metadata
best_text = similar_docs[0][0].page_content
best_score = similar_docs[0][1]

In [ ]:
best_match

In [ ]:
type(best_match)

In [ ]:
# Access the nested metadata dictionary
#metadata = best_match.get('metadata', {})
metadata = best_match

# Retrieve page_num and bbox from the metadata
page_num = metadata.get('page_num')
bbox = metadata.get('bbox')

In [ ]:
page_num

In [ ]:
bbox

In [ ]:
best_text

In [ ]:
if page_num is None or bbox is None:
    print("Metadata is missing 'page_num' or 'bbox'.")
else:
    print(f"Best Match Found: {best_text} with score {best_score} on page {page_num}")

In [ ]:
# Step 7: Highlight the response in the PDF document
page = doc.load_page(page_num)    

In [ ]:
quads = [(bbox[0], bbox[1], bbox[2], bbox[1], bbox[2], bbox[3], bbox[0], bbox[3])]

In [ ]:
page

In [ ]:
page.add_highlight_annot(bbox)

In [ ]:
type(page)

In [ ]:
# Save the highlighted PDF
doc.save(highlighted_pdf_path)

# Conclusion

<ol style="font-size: 20px;">
  <li>PyPDFLoader and ChromaDB extracts the right content. Both Mistral and GPT works well as LLM model. For highlighting text cosine similarity serach works fine </li>
  <li>FAISSDB doesn't work well to get right context. Highlighting based on fitz based metadata gives error.</li>  
</ol>